In [1]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = yf.download('AAPL', start='2018-01-01', end='2023-01-01')
df.reset_index(inplace=True)
df
sg = df.copy()

[*********************100%%**********************]  1 of 1 completed


Candlestick patterns to be implemented: 
1. marubozu
2. hammer
3. hanging man
4. shooting star
5. engulfing pattern

In [2]:
#for checking uptrend and downtrend

df['downtrend']=0

for i in range(10, len(df['Open'])):                                      #starting from day 10 so as to have a downward trend before 
    down=0
    
    for k in range(0,10):
        if df['Close'].iloc[i-k] < df['Open'].iloc[i-k]:
            down+=1
    
    if down>=6:                                                            #atleast 6 out of the past 10 candles to be red and
        df['downtrend'].iloc[i]=1   

df['uptrend']=0
for i in range(10, len(df['Open'])):                                      #starting from day 10 so as to have a uptrend trend before 
    up=0
    
    for k in range(0,10):
        if df['Close'].iloc[i-k] > df['Open'].iloc[i-k]:
            up+=1
    
    if up>=6:                                                            #atleast 6 out of the past 10 candles to be green and
        df['uptrend'].iloc[i]=1  

df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0
...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0


## Marubozu



In [3]:
''' TO CHECK FOR MARAZOU
1. open~high/low
2. close ~low/high
3. no need to check for previous pattern
'''
df['marubozu']='NaN'
def marazou():
    for i in range (10, len(df['Open'])):
        if df['Close'].iloc[i]  >  df['Open'].iloc[i] and ( df['High'].iloc[i]/df['Close'].iloc[i]>=1 and df['High'].iloc[i]/df['Close'].iloc[i]<1.03 ) and ( df['Open'].iloc[i]/df['Low'].iloc[i]>=1 and df['Open'].iloc[i]/df['Low'].iloc[i]<1.05 ) :      #uptrend
           df['marubozu'].iloc[i]=1  #uptrend marazou is assigned as 1
        if df['Close'].iloc[i]  <  df['Open'].iloc[i] and ( df['High'].iloc[i]/df['Open'].iloc[i]>=1 and df['High'].iloc[i]/df['Open'].iloc[i] < 1.03 ) and ( df['Close'].iloc[i]/df['Low'].iloc[i]>=1 and df['Close'].iloc[i]/df['Low'].iloc[i]<1.05 ): #downtrend
            df['marubozu'].iloc[i]=0   #downtrend marazou is assigned as 0

marazou()
df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend,marubozu
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0,NaN
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0,NaN
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0,NaN
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0,NaN
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0,1
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0,0
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0,0
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0,1


## Hammer



In [4]:
'''Conditions for hammer patterns
1. downtrend = 1
2. close/high - low = 2*( high/close - close/high)
3. the hammer formation candle also has a shorter upper wick
applying conditions
'''

df['Hammer'] = 0

for i in range(10, len(df['Open'])):
    if df['Open'].iloc[i] > df['Close'].iloc[i]:
        if  df['downtrend'].iloc[i]==1 and  (df['Close'].iloc[i]-df['Low'].iloc[i])/(df['Open'].iloc[i] -df['Close'].iloc[i] )>2 and (df['High'].iloc[i]-df['Open'].iloc[i]) < 0.5*(df['Open'].iloc[i] - df['Close'].iloc[i]):
            df['Hammer'].iloc[i] = 1
    elif df['Close'].iloc[i] > df['Open'].iloc[i]:
        if  df['downtrend'].iloc[i]==1 and  (df['Open'].iloc[i]-df['Low'].iloc[i])/(df['Close'].iloc[i] -df['Open'].iloc[i] )>2 and (df['High'].iloc[i]-df['Close'].iloc[i]) < 0.5*(df['Close'].iloc[i] - df['Open'].iloc[i]):
            df['Hammer'].iloc[i] = 1

df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend,marubozu,Hammer
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0,NaN,0
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0,NaN,0
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0,NaN,0
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0,NaN,0
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0,1,0
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0,0,0
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0,0,0
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0,1,0


## Hanging Man


In [5]:
df['hanging man']=0

for i in range(10, len(df['Open'])):
    if df['Open'].iloc[i] > df['Close'].iloc[i]:
        if  df['downtrend'].iloc[i]==1 and  (df['Close'].iloc[i]-df['Low'].iloc[i])/(df['Open'].iloc[i] -df['Close'].iloc[i] )>2 and (df['High'].iloc[i]-df['Open'].iloc[i]) < 0.5*(df['Open'].iloc[i] - df['Close'].iloc[i]):
            df['hanging man'].iloc[i] = 1
    elif df['Close'].iloc[i] > df['Open'].iloc[i]:
        if  df['downtrend'].iloc[i]==1 and  (df['Open'].iloc[i]-df['Low'].iloc[i])/(df['Close'].iloc[i] -df['Open'].iloc[i] )>2 and (df['High'].iloc[i]-df['Close'].iloc[i]) < 0.5*(df['Close'].iloc[i] - df['Open'].iloc[i]):
            df['hanging man'].iloc[i] = 1

df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend,marubozu,Hammer,hanging man
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0,NaN,0,0
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0,NaN,0,0
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0,NaN,0,0
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0,NaN,0,0
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0,1,0,0
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0,0,0,0
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0,0,0,0
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0,1,0,0


## Shooting Star


In [6]:
df['shootingstar']=0

for i in range(10, len(df['Open'])):
    if df['Open'].iloc[i] > df['Close'].iloc[i] and df['uptrend'].iloc[i]==1 :
        if (df['High']-df['Close']).iloc[i]/(df['Close']-df['Open']).iloc[i] > 2  and  (df['Close']-df['Open']).iloc[i]/(df['Open']-df['Low']) > 2 :
            df['shootingstar']=1
    if df['Open'].iloc[i] < df['Close'].iloc[i] and df['uptrend'].iloc[i]==1:
        if (df['High']-df['Open']).iloc[i]/(df['Open']-df['Close']).iloc[i] > 2  and   (df['Open']-df['Close']).iloc[i]/(df['Close']-df['Low']) > 2  :  
            df['shootingstar']=1

df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend,marubozu,Hammer,hanging man,shootingstar
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0,NaN,0,0,0
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0,NaN,0,0,0
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0,NaN,0,0,0
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0,NaN,0,0,0
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0,1,0,0,0
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0,0,0,0,0
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0,0,0,0,0
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0,1,0,0,0


## Engulfing pattern


In [7]:
df['engulfing']='NaN'

for i in range(10,len(df['Open'])):
    if df['downtrend'].iloc[i]==1 and (df['Close']-df['Open']).iloc[i-1]<0:                                 #bullish engulfing
        if df['Open'].iloc[i]<df['Close'].iloc[i-1] and df['Close'].iloc[i]>df['Open'].iloc[i-1]:
            df['engulfing'].iloc[i]=1                                                                       #assigning value 1 for bullish 
            
    if df['uptrend'].iloc[i]==1 and (df['Close']-df['Open']).iloc[i-1]>0:                                   #bearish engulfing
        if df['Open'].iloc[i]>df['Close'].iloc[i-1] and df['Close'].iloc[i]<df['Open'].iloc[i-1]:
            df['engulfing'].iloc[i]=0                                                                       #assigning value 0 for bearish


df

,Date,Open,High,Low,Close,Adj Close,Volume,downtrend,uptrend,marubozu,Hammer,hanging man,shootingstar,engulfing
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,0,0,NaN,0,0,0,NaN
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,0,0,NaN,0,0,0,NaN
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,0,0,NaN,0,0,0,NaN
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,0,0,NaN,0,0,0,NaN
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,0,0,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,0,1,0,0,0,NaN
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,1,0,0,0,0,0,NaN
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,1,0,0,0,0,0,NaN
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,0,1,0,0,0,NaN


## Reading signals from Candlestick Patterns

1. bullish marubozu = buy
2. bearish marubozu = sell
3. hammer = buy
4. hanging man = sell
5. shooting star = sell
6. bullish engulf = buy
7. bearish engulf = sell

In [26]:
sg['signal']= 'NaN'

for i in range (10,len(df['Open'])):
    if df['marubozu'].iloc[i]==1 or df['Hammer'].iloc[i]==1 or df['engulfing'].iloc[i]==1:
        sg['signal'].iloc[i]=1
    if df['marubozu'].iloc[i]==0 or df['hanging man'].iloc[i]==1 or df['shootingstar'].iloc[i]==1 or df['engulfing'].iloc[i]==0 :
        sg['signal'].iloc[i]=0

sg    

st1= sg.copy()
# sg.to_excel('signals.xlsx',engine='openpyxl')

# Implementing Buy and Sell Signals


## could not complete the trading strategy because i was travelling

1. first approach- buying at buy signal and selling at sell signal

assumption - Due to the inherent nature of how the signals are calculated, the signal is generated at the end of the day. Thus at the end of the day, if we get a buy signal, then we buy at the open of the next day and when we get a sell signal, then we sell at the closing price of the day itself.

In [31]:
j = 0
current = 1           #stores current holding (bought or not), initiated to bought already
returns = 0
st1['buysell']='-'

for i in range (10,len(df['Open'])):
    #checking for buy
    if sg['signal'].iloc[i-1]==1 and current!=1:
        st1['buysell'].iloc[i]='BUY'
        j = i
        current = 1

    if sg['signal'].iloc[i]==0 and current!=0:
        st1['buysell'].iloc[i]='SELL'
        returns+= sg['Close'].iloc[i]-sg['Open'].iloc[j]
        current = 0

    print(returns)

st1



0
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
2.2749977111816406
0.6824989318847656
0.6824989318847656
0.6824989318847656
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
-0.2050018310546875
2.270000457763672
2.270000457763672
2.270000457763672
2.270000457763672
2.270000457763672
3.450000762939453
3.450000762939453
3.450000762939453
3.450000762939453
3.450000762939453
3.8149986267089844
3.8149986267089844
3.8149986267089844
3.8149986267089844
3.8149986267089844
4.9375
4.9375
4.9375
4.780002593994141
4.780002593994141
4.780002593994141
4.780002593994141
4.780002593994141
4.780002593994141
4.780002593994141
3.4450035095214844
3.4450035095214844
3.4450035095214844
3.4450035095214844
3.6325035095214844
3.6325035095214844
3.6325035095214844
2.582504272460937

,Date,Open,High,Low,Close,Adj Close,Volume,signal,buysell
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,NaN,-
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.715786,118071600,NaN,-
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.904903,89738400,NaN,-
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.370628,94640000,NaN,-
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.216965,82271200,NaN,-
...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,1,-
1255,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,0,SELL
1256,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339417,85438400,0,-
1257,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,1,-


In [33]:
st1.to_excel(r"C:\FACquant\Assg3\buysell.xlsx")